In [ ]:
from sympy import *
from sympy.physics.mechanics import *

# enable LaTeX #printing
init_vprinting()

In [ ]:
t, g, l1, l2, l3, l4, l5, r1, r2, r3, r4, r5, m1, m2, m3, m4, m5, i1, i2, i3, i4, i5 = symbols('t g l_1 l_2 l_3 l_4 l_5 r_1 r_2 r_3 r_4 r_5 m_1 m_2 m_3 m_4 m_5 I_1 I_2 I_3 I_4 I_5')
q1, q2, q3, q4, q5, a = dynamicsymbols(r'\theta_1 \theta_2 \theta_3 \theta_4 \theta_5 \alpha')
pi, g1, g2, g3, g4, g5, La = symbols('\pi G_1 G_2 G_3 G_4 G_5 L')

In [ ]:
a = (Rational(1,2) * pi) + q5 + q2 + q3

q1_d = diff(q1, t)
q2_d = diff(q2, t)
q3_d = diff(q3, t)
q4_d = diff(q4, t)
q5_d = diff(q5, t)
q1_dd = diff(q1_d, t)
q2_dd = diff(q2_d, t)
q3_dd = diff(q3_d, t)
q4_dd = diff(q4_d, t)
q5_dd = diff(q5_d, t)


In [ ]:
def get_rot_mat(theta):
    
    cth = cos(theta)
    sth = sin(theta)
    rot_mat = Matrix([[cth, -1*sth], [sth, cth]])
    
    return rot_mat

In [ ]:
A0 = get_rot_mat(a)
A1 = get_rot_mat(q1)
A2 = get_rot_mat(q2)
A3 = get_rot_mat(q3)
A4 = get_rot_mat(q4)
A5 = get_rot_mat(q5)

In [ ]:
parts = {'tibia1': 1, 'femur1': 2, 'torso': 3, 'femur2': 4, 'tibia2': 5}
part_maping = {0: 0, 1: 3, 2: 2, 3: 5, 4: 1, 5: 4} 

property_maping = {
                   0: [0, A0, 0, 0, 0, 0],1: [q1, A1, l1, m1, i1, q1_d, g1], 2: [q2, A2, l2, m2, i2, q2_d, g2], 
                   3: [q3, A3, l3, m3, i3, q3_d, g3], 4: [q4, A4, l4, m4, i4, q4_d, g4], 5: [q5, A5, l5, m5, i5, q5_d, g5]
                  }

In [ ]:
def get_path(where):
    
    part = parts[where]
    path = []

    for i in range(part, 0, -1):
        if part > 3:
            if i != 3:
                path.append(i)
        else:
            path.append(i)
            
    return path

In [ ]:
def get_locations(path_part_map):
    
    locations = [l1, l2, l3, l4, l5]
    locations_cg = [r1, l2-r2, l3-r3, r4, r5]
    
    replace_cg_index = path_part_map[0]-1
    locations[replace_cg_index] = locations_cg[replace_cg_index]
    
    final_loc = [locations[i-1] for i in path_part_map]
    
    return final_loc

In [ ]:
def get_path_part_map(where):
    
    path = get_path(where)
    part_maping = {0: 0, 1: 3, 2: 2, 3: 5, 4: 1, 5: 4} 
    path_part_map = [part_maping[i] for i in path]
    
    return path_part_map

In [ ]:
def get_link_length(part):
    return [property_maping[part][2]], [0]

In [ ]:
def get_part_coord(where):
    
    part = parts[where]
    path = get_path(where)
    
    path_part_map = get_path_part_map(where)
    cg_locations = get_locations(path_part_map)
    
    rot_mat = A0
    coord = Matrix([[0], [0]])
    local_frame = coord
    
    for i in range(len(path_part_map)-1, -1, -1):
        coord += rot_mat * Matrix(get_link_length(path_part_map[i]))
        rot_mat *= property_maping[i+1][1]
#         #print("i: ", i, "\t part: ", path_part_map[i])
    
    coord += rot_mat*local_frame
    
    return coord

In [ ]:
def get_coord():
    for part, id in parts.items():
#     #print(property_maping[part_maping[id]][6])
        property_maping[part_maping[id]].append(get_part_coord(part))

In [ ]:
def get_v_squared(coord):
    
    #print('finding diffs')
    x_d_sq = diff(coord[0], t)**2
    y_d_sq = diff(coord[1], t)**2
    
    #print('adding diffs')
    v_2 = x_d_sq + y_d_sq
    
    return v_2

In [ ]:
def get_langrange():
    
    T = 0
    V = 0
    
    for i in range(1, len(property_maping)):
        
        #print('in  index: ', i)
        
        mass = property_maping[i][3]
        inertia = property_maping[i][4]
        position_vector = property_maping[i][7]
        angle_d_sq = property_maping[i][5]**2
        
        #print('\tfinding v_sq')
        v_sq = get_v_squared(position_vector)
        
        #print('\tAdding to energies')
        #print('\t\tNow T')
        T += Rational(1,2) * mass * v_sq
        T += Rational(1,2) * inertia * angle_d_sq
        #print('\t\tNow V')
        V += mass * g * position_vector[1]
    
    #print('Lagrange - finding')
    L = T-V
    #print('L is done simplifying now...')
    return L

In [ ]:
def get_equations_of_motion(L, q, q_d):
    LE = diff(L, q) - diff(diff(L, q_d), t)
    return LE

In [ ]:
def lagrange():
    
    #print('getting the generalised coordinates now...')
    get_coord()
    #print('getting lagrange now...')
    L = get_langrange()
    La = L
    for i in range(1, len(property_maping)):
        LEi = get_equations_of_motion(L, property_maping[i][0], property_maping[i][5])
        property_maping[i].append(LEi)
        #print("Done for index: ", i)

In [ ]:
def solve_for_dds():
    LE = [property_maping[i][8] for i in range(1, len(property_maping))]
    sols = solve([LE[0], LE[1], LE[2], LE[3], LE[4]], (q1_dd, q2_dd, q3_dd, q4_dd, q5_dd),
                simplify=False, rational=False)
    return sols

In [ ]:
def get_subs(r_c = 'c', c = 0):
    substitutions = {q1_dd: 0, q2_dd: 0, q3_dd: 0, q4_dd: 0, q5_dd: 0, q1_d : 0, q2_d : 0, q3_d : 0, q4_d : 0, q5_d : 0, q1 : 0, q2 : 0, q3 : 0, q4 : 0, q5 : 0}
    if r_c == 'c':
        if c == 1:
            substitutions[q1_dd] = 1
        elif c == 1:
            substitutions[q2_dd] = 1
        elif c == 1:
            substitutions[q3_dd] = 1
        elif c == 1:
            substitutions[q4_dd] = 1
        elif c == 1:
            substitutions[q5_dd] = 1
    return substitutions

mass_matrix = []
c = []

def get_mass_matrix_and_c_matrix():
    for i in range(1, len(property_maping)):
        LE = property_maping[i][8]
        mm_row = []
        for j in range(1, len(property_maping)):  
            sub = get_subs('c', j)
            MM = simplify(LE.subs(sub))
            mm_row.append(MM)
        mass_matrix.append(mm_row)
    #print('Mass matrix done')

    for i in range(1, len(property_maping)):
        LE = property_maping[i][8]
        sub = get_subs()
        #print("in index: ", i)
        c_row = [LE.subs(sub)]
        c.append(c_row)

In [ ]:
def call_in_order():
    #get_lagrangian
    lagrange()
    #find the mass matrix and c matrix to get the state space form
    get_mass_matrix_and_c_matrix()